# Homework 5
## MLE
### Unigram model


In [2]:
seq = ['A', 'B', 'A', 'B', 'B', 'C', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'C']


In [3]:
theta_a_op = seq.count('A')/len(seq)
theta_b_op = seq.count('B')/len(seq)
theta_c_op = seq.count('C')/len(seq)

print(f"the thetha optimun for A is: {theta_a_op}")
print(f"the thetha optimun for B is: {theta_b_op}")
print(f"the thetha optimun for C is: {theta_c_op}")

the thetha optimun for A is: 0.42857142857142855
the thetha optimun for B is: 0.35714285714285715
the thetha optimun for C is: 0.21428571428571427


Using the MLE estimate of $\theta$ on $\mathcal{D}$, which of the following sequences is most likely?

In [4]:
seq_1 = theta_a_op*theta_b_op*theta_c_op
seq_2 = theta_b_op*theta_b_op*theta_b_op
seq_3 = theta_a_op*theta_b_op*theta_b_op
seq_4 = theta_a_op*theta_a_op*theta_c_op

print(seq_1, seq_2, seq_3, seq_4)

0.03279883381924198 0.04555393586005831 0.05466472303206997 0.039358600583090375


### Bigram Model 2

Which of the following represents the MLE for the conditional probability $p(w_2 \mid w_1)$

#### Bigram model — basic idea
\begin{equation}
P\left(w_1, \ldots, w_T\right) \approx \prod_{t=1}^T P\left(w_t \mid w_{t-1}\right),
\end{equation}
where by convention $w_0$ is a special start token ⟨s⟩.
So the conditional probability you need to model is $p(w_i \mid w_{i-1})$

\begin{equation}
\hat{P}_{\mathrm{MLE}}\left(w_i \mid w_{i-1}\right)=\frac{c\left(w_{i-1}, w_i\right)}{c\left(w_{i-1}\right)} .
\end{equation}

we obtained $w_{i-1}$ by summing over (i.e. marginalizing out) all possible continuations $w$ in the joint counts $c\left(w_{i-1}, w\right)$

\begin{equation}
c\left(w_{i-1}\right)=\sum_w c\left(w_{i-1}, w\right)
\end{equation}

\begin{equation}
\hat{P}_{\mathrm{MLE}}\left(w_i \mid w_{i-1}\right)=\frac{c\left(w_{i-1}, w_i\right)}{\sum_w c\left(w_{i-1}, w\right)} .
\end{equation}


In [5]:
P_aa = 1/6
P_ba = 4/6
P_cb = 2/5
P_bc = 0

The answer is zero due to the P_bc = 0

## EM Algorithm
### Likelihood Function
Consider the following mixture of two Gaussians:
\begin{equation}
p(x; \theta ) = \pi _1 \mathcal{N}(x; \mu _1, \sigma _1^2) + \pi _2\mathcal{N}(x; \mu _2, \sigma _2^2)
\end{equation}

This mixture has parameters $\theta = \{ \pi _1, \pi _2, \mu _1, \mu _2, \sigma _1^2, \sigma _2^2\}$
We initialize $\theta$ as $\theta _0 = \{ 0.5, 0.5, 6, 7, 1, 4\}$.

We have a dataset $\mathcal{D}$ with the following samples of
\begin{equation}
x: x^{(0)}=-1, x^{(1)}=0, x^{(2)}=4, x^{(3)}=5, x^{(4)}=6 .
\end{equation}



In [6]:
x = [-1, 0, 4, 5, 6]
pi = [0.5, 0.5]
mu = [6, 7]
sigma_2 = [1, 4]

What is the log-likelihood of the data $l(\mathcal{D}; \theta )$ given the initial setting of $\theta$?
\begin{equation}
l(\mathcal{D} ; \theta)=\sum_{i=0}^{N-1} \log \left(\pi_1 \mathcal{N}\left(x^{(i)} ; \mu_1, \sigma_1^2\right)+\pi_2 \mathcal{N}\left(x^{(i)} ; \mu_2, \sigma_2^2\right)\right)
\end{equation}

In [8]:
import numpy as np

def norm_dist(x, mu, sigma_2):
    P = 1/(np.sqrt(2*np.pi*sigma_2)) * np.exp(-1/(2*sigma_2) * (x-mu)**2)
    return P
    
def post_prob(x, p, mu, sigma_2, i, j):
    num_p_j_i = p[j]*norm_dist(x[i], mu[j], sigma_2[j])
    den = p[0]*norm_dist(x[i], mu[0], sigma_2[0]) + p[1]*norm_dist(x[i], mu[1], sigma_2[1])
    post_prob = num_p_j_i/den
    return post_prob

In [10]:
# --- Log-Likelihood Calculation ---

log_likelihood = 0
# Loop through each data point
for i in range(len(x)):
    # Calculate the likelihood for the current data point x[i]
    # This is the sum of probabilities from each Gaussian component
    likelihood_i = pi[0] * norm_dist(x[i], mu[0], sigma_2[0]) + \
                   pi[1] * norm_dist(x[i], mu[1], sigma_2[1])
    
    log_likelihood += np.log(likelihood_i)

print(f"The log-likelihood of the data is: {log_likelihood:.1f}")


The log-likelihood of the data is: -24.5


### E-Step
What is the formula for $p(y = k \mid x, \theta )$? 

\begin{equation}
P\left(y=k \mid x_i, \Theta\right)=\frac{\pi_k \mathcal{N}\left(x \mid \mu_k, \Sigma_k\right)}{\sum_{j=1}^K \pi_j \mathcal{N}\left(x_i \mid \mu_j, \Sigma_j\right)} .
\end{equation}

\begin{equation}
P\left(y=k \mid x_i, \Theta\right)=\frac{\pi_k N_k}{ \pi_1 N_1 + \pi_2 N_2} .
\end{equation}

In [12]:
# x[0]

for i in range(len(x)):
    if post_prob(x, pi, mu, sigma_2, i, 1) > post_prob(x, pi, mu, sigma_2, i, 0):
        print(f"x = {i} Gaussian {2}")
    else:
        print(f"x = {i} Gaussian {1}")

x = 0 Gaussian 2
x = 1 Gaussian 2
x = 2 Gaussian 2
x = 3 Gaussian 1
x = 4 Gaussian 1


### M-Step
Fixing $p(y = k \mid x, \theta _0)$, we want to update $\theta$ such that our lower bound is maximized.

What is the optimal $\hat{\mu }_ k$?

\begin{equation}
N_k=\sum_{i=1}^n \gamma_{i, k}
\end{equation}

\begin{equation}
\pi_k=\frac{N_k}{n}
\end{equation}

\begin{equation}
\mu_k^{\text {new }}=\frac{1}{N_k} \sum_{i=1}^n \gamma_{i, k} x_i .
\end{equation}

\begin{equation}
\hat{\mu_k}=\frac{1}{\sum_{i=1}^n \gamma_{i, k}} \sum_{i=1}^n \gamma_{i, k} x_i .
\end{equation}

\begin{equation}
\hat{\sigma_k} = \frac{1}{\sum_{i=1}^n \gamma_{i, k}} \sum_{i=1}^n \gamma_{i, k}\left(x_i-\hat{\mu_k}\right)\left(x_i-\hat{\mu_k}\right)^T .
\end{equation}

\begin{equation}
\hat{\sigma_k} = \frac{1}{\sum_{i=1}^n \gamma_{i, k}} \sum_{i=1}^n \gamma_{i, k}\left(x_i-\hat{\mu_k}\right)^2 .
\end{equation}

\begin{equation}
\pi_k=\frac{\sum_{i=1}^n \gamma_{i, k}}{n}
\end{equation}


### Training 1
In the first M-step, which Gaussian will shift to the left more (relatively)?

In [16]:
N_1 = 0
for i in range(len(x)):
    N_1 += post_prob(x, pi, mu, sigma_2, i, 0)
print(f"effective cluster 1 size: {N_1}")

N_2 = 0
for i in range(len(x)):
    N_2 += post_prob(x, pi, mu, sigma_2, i, 1)
print(f"effective cluster 1 size: {N_2}")

sum_mu = 0
for i in range(len(x)):
    sum_mu += post_prob(x, pi, mu, sigma_2, i, 0) * x[i]
mu_1_new = sum_mu/N_1 
print(f"New mean 1: {mu_1_new}")

sum_mu = 0
for i in range(len(x)):
    sum_mu += post_prob(x, pi, mu, sigma_2, i, 1) * x[i]
mu_2_new = sum_mu/N_2 

print(f"Old mean 1: {mu[0]} - New mean 1: {mu_1_new}")
print(f"Old mean 2: {mu[1]} - New mean 2: {mu_2_new}")

effective cluster 1 size: 1.8151852969722184
effective cluster 1 size: 3.1848147030277816
New mean 1: 5.131728029552676
Old mean 1: 6 - New mean 1: 5.131728029552676
Old mean 2: 7 - New mean 2: 1.4710314946241008


Answer mean 2

### Training 2
In the first M-step, which Gaussian's variance will increase more (relatively)?



In [18]:
sum_sigma = 0
for i in range(len(x)):
    sum_sigma += post_prob(x, pi, mu, sigma_2, i, 0) * (x[i] - mu_1_new)**2
sigma_2_1_new = sum_sigma/N_1 

sum_sigma = 0
for i in range(len(x)):
    sum_sigma += post_prob(x, pi, mu, sigma_2, i, 1) * (x[i] - mu_2_new)**2
sigma_2_2_new = sum_sigma/N_2 


print(f"Old variance 1: {sigma_2[0]} - New variance 1: {sigma_2_1_new}")
print(f"Old variance 2: {sigma_2[1]} - New variance 2: {sigma_2_2_new}")

Old variance 1: 1 - New variance 1: 0.615562501441774
Old variance 2: 4 - New variance 2: 6.967022101798352


In [46]:
def compute_cluster_size(x, pi, mu, sigma_2, k):
    N_k = 0
    for i in range(len(x)):
        N_k += post_prob(x, pi, mu, sigma_2, i, k)
    return N_k.item()

def update_pi(x, N_k):
    pi_k = N_k/len(x)
    return pi_k
    
def update_mu(x, pi, mu, sigma_2, k, N_k):
    sum_mu = 0
    for i in range(len(x)):
        sum_mu += post_prob(x, pi, mu, sigma_2, i, k) * x[i]
    mu_k_new = sum_mu/N_k 

    return mu_k_new.item()

def update_sigma_2(x, pi, mu, sigma_2, k, N_k, mu_k_new):
    sum_sigma = 0
    for i in range(len(x)):
        sum_sigma += post_prob(x, pi, mu, sigma_2, i, 0) * (x[i] - mu_k_new)**2
        sigma_2_k_new = sum_sigma/N_k 
    return sigma_2_k_new.item()

In [47]:
pi_array = [pi]
mu_array = [mu]
sigma_2_array = [sigma_2]

print(pi_array, mu_array, sigma_2_array)

[[0.5, 0.5]] [[6, 7]] [[1, 4]]


In [48]:
for epoch in range(10):
    cluster_size_j = []
    pi_k = []
    mu_k = []
    sigma_2_k = []
    for k in range(2):
        cluster_size_j.append(compute_cluster_size(x, pi_array[epoch], mu_array[epoch], sigma_2_array[epoch], k))
        pi_k.append(update_pi(x, cluster_size_j[k]))
        mu_k.append(update_mu(x, pi_array[epoch], mu_array[epoch], sigma_2_array[epoch], k, cluster_size_j[k]))
        sigma_2_k.append(update_sigma_2(x, pi_array[epoch], mu_array[epoch], sigma_2_array[epoch], k, cluster_size_j[k], mu_k[k]))
    
    
    pi_array.append(pi_k)
    mu_array.append(mu_k)
    sigma_2_array.append(sigma_2_k)

 

In [51]:
sigma_2_array = np.array(sigma_2_array)
print(np.round(sigma_2_array, 2))

[[ 1.    4.  ]
 [ 0.62  7.99]
 [ 0.6  12.8 ]
 [ 0.62 16.11]
 [ 0.63 18.45]
 [ 0.64 19.91]
 [ 0.64 20.73]
 [ 0.64 21.16]
 [ 0.64 21.37]
 [ 0.64 21.48]
 [ 0.64 21.53]]


No se por qué dice que la varianza 1 es la que mas se incrementa